### <center> 🪙Crypto Price Interactive Dashboard(Daily Update) </center>


In [ ]:
START_DATE = "2024-01-11" # start date для ввода даты
RSI_TIME_WINDOW = 7 #количество дней


import requests              # для HTTP-запросов к API криптобирж
import pandas as pd          # для работы с табличными данными
import warnings              # для управления предупреждениями
import datetime as dt        # для работы с датами и временем
import plotly.express as px  # для создания интерактивных графиков
import matplotlib.pyplot as plt  # для статических графиков (резерв)
import plotly.graph_objects as go  # для низкоуровневого создания графиков
import pandas_datareader.data as web  # для получения финансовых данных
from plotly.subplots import make_subplots  # для создания сложных layouts графиков
warnings.filterwarnings('ignore')  # игнорирование предупреждений

## адреса и названия
urls = ["https://www.cryptodatadownload.com/cdd/Bitfinex_EOSUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_EDOUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_BTCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETHUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_LTCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_BATUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_OMGUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_DAIUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETPUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_NEOUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_REPUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_TRXUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XLMUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XMRUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XVGUSD_d.csv",
       ]
crypto_names = ["EOS Coin (EOS)",# человеко-читаемые названия криптовалют
                "Eidoo (EDO)",
                "Bitcoin (BTC)",
                "Ethereum (ETH)",
                "Litecoin (LTC)",
                "Basic Attention Token (BAT)",
                "OmiseGO (OMG)",
                "Dai (DAI)",
                "Ethereum Classic (ETC)",
                "Metaverse (ETP)",
                "Neo (NEO)",
                "Augur (REP)",
                "TRON (TRX)",
                "Stellar (XLM)",
                "Monero (XMR)",
                "Verge (XVG)"
               ]

 # Загрузчик данных: скачивает CSV файлы, объединяет и очищает данные
def df_loader(urls , start_date = "2021-01-01"):
    filenames = []
    all_df = pd.DataFrame()
    for idx,url in enumerate(urls):
        req = requests.get(url,verify=False)
        url_content = req.content
        filename = url[48:]
        csv_file = open( filename , 'wb')
        csv_file.write(url_content)
        csv_file.close()
        filename = filename[:-9]
        filenames.append(filename)
    for file in filenames:
        df = pd.read_csv(file + "USD_d.csv", header = 1, parse_dates=["date"])
        df = df [df["date"] > start_date]
        df.index = df.date
        df.drop(labels = [df.columns[0],df.columns[1],df.columns[8]] , axis = 1 , inplace = True)
        all_df = pd.concat([all_df,df], ignore_index=False)

    return all_df , filenames
def computeRSI (data, time_window):                                       # Вычисляет индекс относительной силы (RSI)
    diff = data.diff(1).dropna()
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

all_df , filenames = df_loader(urls , start_date=START_DATE)              # загрузка и подготовка данных

crypto_df = []                                                            # список для хранения DataFrame по каждой криптовалюте
for file in filenames:                                                    # обработка каждой криптовалюты
    symbol = file + str("/USD") # Фильтрация данных по символу валютной пары
    temp_df = pd.DataFrame(all_df[all_df["symbol"] == symbol]) # Удаление ненужных столбцов
    temp_df.drop(columns= ["symbol"] ,inplace = True)
    temp_df["close_rsi"] = computeRSI(temp_df['close'], time_window=RSI_TIME_WINDOW) # Расчет RSI для цен закрытия
    temp_df["high_rsi"] = 30  # Добавление уровней RSI (30 и 70)
    temp_df["low_rsi"] = 70
    exec('%s = temp_df.copy()' % file.lower())  # Динамическое создание переменных для каждой криптовалюты
    crypto_df.append(temp_df)

## основа
fig = make_subplots(rows=3,   # создание комплексной сетки графиков
                    cols=2,
                    shared_xaxes=True,
                    specs=[[{"rowspan": 2}, {"rowspan": 2}], [{"rowspan": 1}, {"rowspan": 1}] , [{},{}]]
# кнопки для выбора временных диапазонов
                   )
date_buttons = [
{'step': "all", 'label': "All time"},
{'count':  1, 'step': "year", 'stepmode': "backward", 'label': "Last Year"},
{'count':  1, 'step': "year", 'stepmode': "todate", 'label': "Current Year"},
{'count':  1, 'step': "month", 'stepmode': "backward", 'label': "Last 2 Months"},
{'count':  1, 'step': "month", 'stepmode': "todate", 'label': "Current Month"},
{'count':  7, 'step': "day", 'stepmode': "todate", 'label': "Current Week"},
{'count':  4, 'step': "day", 'stepmode': "backward", 'label': "Last 4 days"},
{'count':  1, 'step': "day", 'stepmode': "backward", 'label': "Today"},
               ]
buttons = [] # кнопки для переключения между криптовалютами
i = 0
j=0
COUNT = 8
vis = [False] * len(crypto_names) * COUNT
for df in crypto_df:
    for k in range(COUNT):
        vis[j+k] = True
    buttons.append({ 'label' : crypto_names[i],
                     'method' : 'update',
                     'args'   : [{'visible' : vis},
                                {'title'  : crypto_names[i] + ' Charts and Indicators'}
                                ] }
                  )
    i+=1
    j+=COUNT
    vis = [False] * len(crypto_names)* COUNT
for df in crypto_df: # для каждого DataFrame криптовалюты
    fig.add_trace(
        go.Candlestick(x= df.index, # Candlestick chart - японские свечи (ряд 1, колонка 1)
                       open  = df['open'],
                       high  = df['high'],
                       low   = df['low'],
                       close = df['close'],
                       showlegend =False
                                ),
                  row=1,
                  col=1)
    fig.add_trace(
        go.Bar(x =df.index ,
               y=df["Volume USD"],
               showlegend =False,
               marker_color='aqua'),
        row=3,
        col=1)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['close'],
                   mode='lines',
                  showlegend =False,
                   line=dict(color="red", width=4)),
        row=1,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['low'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='pink', dash='dash')),
        row=1,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['high'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='pink', dash='dash')),
        row=1,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['close_rsi'],
                   mode='lines',
                  showlegend =False,
                   line=dict(color="aquamarine", width=4)),
        row=3,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['low_rsi'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='aqua', dash='dash')),
        row=3,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['high_rsi'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='aqua', dash='dash')),
        row=3,
        col=2)

# настройка осей X
fig.update_xaxes(
        tickfont = dict(size=15, family = 'monospace', color='#B8B8B8'),
        tickmode = 'array',
        ticklen = 6,
        showline = False,
        showgrid = True,
        gridcolor = '#595959',
        ticks = 'outside')
fig.update_layout(   # основное оформление дашборда
    spikedistance=100,
    xaxis_rangeslider_visible=False,
    hoverdistance=1000)
fig.update_xaxes(
    showspikes=True,
    spikesnap="cursor",
    spikemode="across"
                )
fig.update_yaxes( # настройка осей Y
    showspikes=True,
    spikesnap='cursor',
    spikemode="across"
                )
fig.update_yaxes(
        tickfont = dict(size=15, family = 'monospace', color='#B8B8B8'),
        tickmode = 'array',
        showline = False,
        ticksuffix = '$',
        showgrid = True,
        gridcolor = '#595959',
        ticks = 'outside')
fig.update_layout(
                  width=1120,
                  height=650,
                  font_family   = 'monospace',
                  xaxis         = dict(rangeselector = dict(buttons = date_buttons)),
                  updatemenus   = [dict(type = 'dropdown',
                                        x = 1.0,
                                        y = 1.108,
                                        showactive = True,
                                        active = 2,
                                        buttons = buttons)],
                  title         = dict(text = '<b>Cryptocurrencies  Dashboard<b>',
                                       font = dict(color = '#FFFFFF' ,size = 22),
                                       x = 0.50),
                  font          = dict(color="blue"),
                  annotations   = [
                                  dict( text = "<b>Choose Cryptocurrency: <b>",
                                        font = dict(size = 20 , color = "#ffffff"),
                                        showarrow=False,
                                        x = 1.02,
                                        y = 1.20,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Candlestick Chart <b>",# Candlestick chart - японские свечи (ряд 1, колонка 1)
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.12,
                                        y = 0.285,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Price Chart<b>", # Price chart - линейный график цены (ряд 1, колонка 2)
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.82,
                                        y = 0.285,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Volume Traded<b>",
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.14,
                                        y = -0.53,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Relative Strength Index (RSI)<b>",
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.94,
                                        y = -0.53,
                                        xref = 'paper', yref = "paper",
                                        align = "left")
                  ],
                template= "plotly_dark"
                )
for i in range(0,16*COUNT):
    fig.data[i].visible = False
for i in range(COUNT):
    fig.data[i].visible = True
fig.layout["xaxis"]["rangeslider"]["visible"] = False
fig.layout["xaxis2"]["rangeslider"]["visible"] = False
fig.layout["xaxis5"]["rangeslider"]["visible"] = True
fig.layout["xaxis6"]["rangeslider"]["visible"] = True
fig.layout["xaxis5"]["rangeslider"]["borderwidth"] = 4
fig.layout["xaxis6"]["rangeslider"]["borderwidth"] = 4
fig.layout["xaxis5"]["rangeslider"]["bordercolor"] = "aqua"
fig.layout["xaxis6"]["rangeslider"]["bordercolor"] = "aqua"
fig.layout["yaxis6"]["ticksuffix"] = ""
fig.layout["yaxis6"]["range"] = [10,100]
fig.show()# вывод интерактивного дашборда

Нажмите значок 🏠 в правом верхнем углу, чтобы сбросить настройки панели.

Дата начала исторических данных — 1 января 2024 года.

Всего на панели используется 16 криптовалют.

Период индекса относительной силы (RSI) составляет 7 дней.


Этот код на Python выполняет загрузку, обработку и визуализацию данных о криптовалютах. Вот основные его компоненты и функции:

**1. Определение параметров**

START_DATE: Начальная дата для загрузки исторических данных (11 января 2024 года).
RSI_TIME_WINDOW: Период для расчета индикатора относительной силы (RSI), установленный на 7 дней.

**2. Импорт библиотек**

Код использует несколько библиотек:

requests: Для загрузки данных из интернета.
pandas: Для обработки и анализа данных.
plotly: Для создания интерактивных графиков.
matplotlib: Для визуализации.
pandas_datareader: Для доступа к финансовым данным.

**3. Загрузка данных**

Функция df_loader:

Загружает данные о криптовалютах из указанных URL.
Сохраняет их в CSV-файлы и объединяет в один DataFrame.
Фильтрует данные по дате.

**4. Расчет RSI**
Функция computeRSI:
Рассчитывает RSI на основе закрывающих цен криптовалют.
Использует экспоненциальное скользящее среднее для вычисления средних изменений.

**5. Подготовка данных для графиков**
Создается список DataFrame для каждой криптовалюты, в который добавляется столбец с RSI и установлены уровни для низкого и высокого RSI (30 и 70 соответственно).

**6. Построение графиков**
Используется make_subplots для создания многопанельного графика, который включает в себя:
Свечные графики для цен.
Графики объема торгов.
Графики цен с линиями RSI.

**7. Настройка графиков**
Добавляются кнопки для выбора временного диапазона и криптовалюты.
Настраиваются оси и внешний вид графиков (цвета, размеры шрифтов и т.д.).
Устанавливаются видимость ползунков диапазона для осей X.

**8. Отображение графика**
В конце код вызывает fig.show(), чтобы отобразить интерактивный график.
Заключение
Этот код позволяет загружать данные о различных криптовалютах, рассчитывать их RSI и визуализировать информацию в удобном формате, что полезно для анализа рынка. Если вам нужно больше информации по какому-либо из пунктов, дайте знать!



In [ ]:
%%html
<!-- draw.io diagram -->
<div class="mxgraph" style="max-width:100%;border:1px solid transparent;" data-mxgraph="{&quot;highlight&quot;:&quot;#0000ff&quot;,&quot;nav&quot;:true,&quot;resize&quot;:true,&quot;dark-mode&quot;:&quot;auto&quot;,&quot;toolbar&quot;:&quot;zoom layers tags lightbox&quot;,&quot;edit&quot;:&quot;_blank&quot;,&quot;xml&quot;:&quot;&lt;mxfile host=\&quot;app.diagrams.net\&quot; agent=\&quot;Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36\&quot; version=\&quot;28.1.2\&quot;&gt;\n  &lt;diagram name=\&quot;Страница — 1\&quot; id=\&quot;NVj4Mm-6WkWIkGBEAEDB\&quot;&gt;\n    &lt;mxGraphModel dx=\&quot;1426\&quot; dy=\&quot;785\&quot; grid=\&quot;1\&quot; gridSize=\&quot;10\&quot; guides=\&quot;1\&quot; tooltips=\&quot;1\&quot; connect=\&quot;1\&quot; arrows=\&quot;1\&quot; fold=\&quot;1\&quot; page=\&quot;1\&quot; pageScale=\&quot;1\&quot; pageWidth=\&quot;827\&quot; pageHeight=\&quot;1169\&quot; math=\&quot;0\&quot; shadow=\&quot;0\&quot;&gt;\n      &lt;root&gt;\n        &lt;mxCell id=\&quot;0\&quot; /&gt;\n        &lt;mxCell id=\&quot;1\&quot; parent=\&quot;0\&quot; /&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-1\&quot; value=\&quot;&amp;lt;strike&amp;gt;басова&amp;lt;/strike&amp;gt;\&quot; style=\&quot;rounded=0;whiteSpace=wrap;html=1;fillColor=#e1d5e7;strokeColor=#9673a6;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;695\&quot; y=\&quot;20\&quot; width=\&quot;120\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-5\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;entryX=0.5;entryY=0;entryDx=0;entryDy=0;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-6\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-9\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;390\&quot; y=\&quot;1450\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-6\&quot; value=\&quot;начало&amp;amp;nbsp;\&quot; style=\&quot;ellipse;whiteSpace=wrap;html=1;rounded=0;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;330\&quot; y=\&quot;1350\&quot; width=\&quot;120\&quot; height=\&quot;50\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-31\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-9\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-29\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-9\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Задать&amp;amp;nbsp;параметры:&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;START_DATE,RSI_TIME_WINDOW&amp;lt;/pre&amp;gt;\&quot; style=\&quot;shape=parallelogram;perimeter=parallelogramPerimeter;whiteSpace=wrap;html=1;fixedSize=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;173.75\&quot; y=\&quot;1420\&quot; width=\&quot;432.5\&quot; height=\&quot;50\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-15\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;entryX=0.5;entryY=0;entryDx=0;entryDy=0;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-12\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-38\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;390\&quot; y=\&quot;1650\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-12\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Определить&amp;amp;nbsp;URL&amp;amp;nbsp;и&amp;amp;nbsp;названия &amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;криптовалют&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;\&quot; style=\&quot;whiteSpace=wrap;html=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;222.5\&quot; y=\&quot;1580\&quot; width=\&quot;335\&quot; height=\&quot;45\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-17\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=0.5;exitY=1;exitDx=0;exitDy=0;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-38\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-16\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;390\&quot; y=\&quot;1760\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-19\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-16\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-18\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-16\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Для&amp;amp;nbsp;каждого&amp;amp;nbsp;файла:&amp;amp;nbsp;-&amp;amp;nbsp;Рассчитать&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;RSI&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Создать&amp;amp;nbsp;DataFrame&amp;amp;nbsp;для&amp;amp;nbsp;каждой&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;криптовалюты&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;\&quot; style=\&quot;whiteSpace=wrap;html=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;193.75\&quot; y=\&quot;1785\&quot; width=\&quot;392.5\&quot; height=\&quot;110\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-21\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-18\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-20\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-18\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Создать&amp;amp;nbsp;графики:-&amp;amp;nbsp;Свечные&amp;amp;nbsp;графики&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;-&amp;amp;nbsp;Графики&amp;amp;nbsp;объемов&amp;amp;nbsp;-&amp;amp;nbsp;Графики&amp;amp;nbsp;RSI&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;\&quot; style=\&quot;whiteSpace=wrap;html=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;72.5\&quot; y=\&quot;1920\&quot; width=\&quot;635\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-23\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-20\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-22\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-20\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Настроить&amp;amp;nbsp;графики:&amp;amp;nbsp;-&amp;amp;nbsp;Кнопки&amp;amp;nbsp;для&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;выбора&amp;amp;nbsp;валюты-&amp;amp;nbsp;Настройка&amp;amp;nbsp;осей&amp;amp;nbsp;-&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Оформление&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;\&quot; style=\&quot;whiteSpace=wrap;html=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;42.5\&quot; y=\&quot;2010\&quot; width=\&quot;695\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-26\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-22\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-25\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-22\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Отобразить&amp;amp;nbsp;график&amp;amp;nbsp;с&amp;amp;nbsp;помощью&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;fig.show()&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;\&quot; style=\&quot;shape=parallelogram;perimeter=parallelogramPerimeter;whiteSpace=wrap;html=1;fixedSize=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;197.5\&quot; y=\&quot;2110\&quot; width=\&quot;385\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-25\&quot; value=\&quot;Конец\&quot; style=\&quot;ellipse;whiteSpace=wrap;html=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;297.5\&quot; y=\&quot;2210\&quot; width=\&quot;185\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-34\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;entryX=0.5;entryY=0;entryDx=0;entryDy=0;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-29\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-12\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-29\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Импортировать&amp;amp;nbsp;библиотеки&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;requests,&amp;amp;nbsp;pandas,&amp;amp;nbsp;plotly&amp;amp;nbsp;и&amp;amp;nbsp;др.)&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;\&quot; style=\&quot;shape=parallelogram;perimeter=parallelogramPerimeter;whiteSpace=wrap;html=1;fixedSize=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;132.5\&quot; y=\&quot;1500\&quot; width=\&quot;515\&quot; height=\&quot;50\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-35\&quot; value=\&quot;Actor\&quot; style=\&quot;shape=umlActor;verticalLabelPosition=bottom;verticalAlign=top;html=1;outlineConnect=0;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;30\&quot; y=\&quot;1100\&quot; width=\&quot;10\&quot; height=\&quot;30\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-38\&quot; value=\&quot;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Вызвать&amp;amp;nbsp;функцию&amp;amp;nbsp;df_loader()&amp;amp;nbsp;и&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;загрузить&amp;amp;nbsp;данные:&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;&amp;lt;pre style=&amp;quot;margin: 1em 0px; color: rgb(31, 31, 31); background-color: rgb(247, 247, 247); border-radius: 2px; font-size: 14px; padding: 12px 8px; overflow-x: auto; text-align: start;&amp;quot;&amp;gt;&amp;lt;code&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;-&amp;amp;nbsp;Загрузить&amp;amp;nbsp;CSV&amp;amp;nbsp;-&amp;amp;nbsp;&amp;lt;/span&amp;gt;&amp;lt;span style=&amp;quot;color: rgb(0, 0, 0);&amp;quot; class=&amp;quot;mtk1&amp;quot;&amp;gt;Объединить&amp;amp;nbsp;данные -&amp;amp;nbsp;Отфильтровать&amp;amp;nbsp;по&amp;amp;nbsp;дате&amp;lt;/span&amp;gt;&amp;lt;/code&amp;gt;&amp;lt;/pre&amp;gt;\&quot; style=\&quot;shape=parallelogram;perimeter=parallelogramPerimeter;whiteSpace=wrap;html=1;fixedSize=1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;130\&quot; y=\&quot;1650\&quot; width=\&quot;520\&quot; height=\&quot;110\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-41\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-39\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-40\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-39\&quot; value=\&quot;Начало\&quot; style=\&quot;ellipse;whiteSpace=wrap;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;300\&quot; y=\&quot;20\&quot; width=\&quot;180\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-43\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-40\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-42\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-40\&quot; value=\&quot;Инициализация параметров:&amp;#xa;START_DATE = &amp;quot;2024-01-11&amp;quot;RSI_TIME_WINDOW = 7\&quot; style=\&quot;ellipse;whiteSpace=wrap;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;267.5\&quot; y=\&quot;110\&quot; width=\&quot;245\&quot; height=\&quot;90\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-47\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-42\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-46\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-42\&quot; value=\&quot;Импорт библиотек\&quot; style=\&quot;ellipse;whiteSpace=wrap;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;312.5\&quot; y=\&quot;220\&quot; width=\&quot;155\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-49\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-46\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-48\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-46\&quot; value=\&quot;Определение данных&amp;#xa;URLs массив адресов данныхcrypto_names массив названий криптовалют\&quot; style=\&quot;shape=parallelogram;perimeter=parallelogramPerimeter;whiteSpace=wrap;fixedSize=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;230\&quot; y=\&quot;300\&quot; width=\&quot;320\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-51\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-48\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-50\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-48\&quot; value=\&quot;Функция df_loader():&amp;#xa;Загрузка данных с указанных URL&amp;#xa;Сохранение в CSV файлы&amp;#xa;Объединение данных в единый DataFrame&amp;#xa;Фильтрация по START_DATE\&quot; style=\&quot;shape=parallelogram;perimeter=parallelogramPerimeter;whiteSpace=wrap;fixedSize=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;160\&quot; y=\&quot;380\&quot; width=\&quot;460\&quot; height=\&quot;90\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-53\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-50\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-52\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-50\&quot; value=\&quot;Функция computeRSI():&amp;#xa;Вычисление индекса относительной силы&amp;#xa;Расчет разницы цен&amp;#xa;Определение восходящих и нисходящих изменений&amp;#xa;Вычисление скользящих средних&amp;#xa;Расчет RSI значения\&quot; style=\&quot;whiteSpace=wrap;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;200\&quot; y=\&quot;500\&quot; width=\&quot;380\&quot; height=\&quot;100\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-56\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-52\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-55\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-52\&quot; value=\&quot;Обработка данных&amp;#xa;Для каждой криптовалюты:&amp;#xa;Фильтрация данных по символу&amp;#xa;Вычисление RSI для закрытия&amp;#xa;Установка уровней RSI (30 и 70)\&quot; style=\&quot;rhombus;whiteSpace=wrap;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;265\&quot; y=\&quot;630\&quot; width=\&quot;250\&quot; height=\&quot;200\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-58\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;entryX=0.5;entryY=0;entryDx=0;entryDy=0;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-55\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-61\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;390\&quot; y=\&quot;940\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-55\&quot; value=\&quot;Создание визуализации&amp;#xa;Создание subplots (3 строки, 2 столбца)Настройка кнопок выбора датыСоздание кнопок выбора криптовалюты\&quot; style=\&quot;whiteSpace=wrap;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;240\&quot; y=\&quot;850\&quot; width=\&quot;300\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-60\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;exitX=0.5;exitY=1;exitDx=0;exitDy=0;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-61\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-59\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;390\&quot; y=\&quot;1040\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-64\&quot; value=\&quot;\&quot; style=\&quot;edgeStyle=orthogonalEdgeStyle;rounded=0;orthogonalLoop=1;jettySize=auto;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; edge=\&quot;1\&quot; parent=\&quot;1\&quot; source=\&quot;oBxJexhrVtP6csgbW6TP-59\&quot; target=\&quot;oBxJexhrVtP6csgbW6TP-63\&quot;&gt;\n          &lt;mxGeometry relative=\&quot;1\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-59\&quot; value=\&quot;Настройка оформления&amp;#xa;Обновление осей X и YНастройка шрифтов и цветовДобавление заголовков и аннотацийУстановка темного шаблона\&quot; style=\&quot;whiteSpace=wrap;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;230\&quot; y=\&quot;1020\&quot; width=\&quot;320\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-61\&quot; value=\&quot;Добавление графиков&amp;lt;br style=&amp;quot;padding: 0px; margin: 0px;&amp;quot;&amp;gt;Для каждого DataFrame криптовалюты\&quot; style=\&quot;shape=parallelogram;perimeter=parallelogramPerimeter;whiteSpace=wrap;html=1;fixedSize=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;275\&quot; y=\&quot;940\&quot; width=\&quot;230\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;oBxJexhrVtP6csgbW6TP-63\&quot; value=\&quot;Конец\&quot; style=\&quot;ellipse;whiteSpace=wrap;html=1;fillColor=#d0cee2;strokeColor=#56517e;shadow=0;strokeWidth=2;opacity=70;dashed=1;dashPattern=1 1;\&quot; vertex=\&quot;1\&quot; parent=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;325\&quot; y=\&quot;1100\&quot; width=\&quot;130\&quot; height=\&quot;60\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n      &lt;/root&gt;\n    &lt;/mxGraphModel&gt;\n  &lt;/diagram&gt;\n&lt;/mxfile&gt;\n&quot;}"></div>
<script type="text/javascript" src="https://viewer.diagrams.net/js/viewer-static.min.js"></script>
